In [ ]:
"""
Created on Fri Jul 09 09:39 2021

This script is to transform the mean profiles to 2D fields to accelerate the tuning process

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
import multimelt.melt_functions as meltf
from scipy import stats
from dask import delayed

import distributed
import glob

from tqdm.notebook import tqdm


In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=8,dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

READ IN DATA

In [ ]:
mod = 'ACCESS-CM2' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
    to2300 = False
elif mod in ['GISS-E2-1-H']:
    to2300 = True
elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
    to2300 = True
elif mod in ['MPI-ESM1-2-HR','CESM2','GFDL-CM4','GFDL-ESM4']:
    to2300 = False
elif mod == 'UKESM1-0-LL':
    to2300 = True        

if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100        

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_mask='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 600
Nisf_chunk = 30

Files for param

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_mfdataset(inputpath_isf+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf).chunk(chunks={'x': chunk_size, 'y': chunk_size})


In [ ]:
BedMachine_orig = xr.open_mfdataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
BedMachine_orig_cut = BedMachine_orig.sel(x=file_isf.x, y=file_isf.y).chunk(chunks={'x': chunk_size, 'y': chunk_size})

ice_draft_pos = (BedMachine_orig_cut['thickness'] - BedMachine_orig_cut['surface']).where(file_isf['ISF_mask'] > 1)
ice_draft_isf = ice_draft_pos.where(file_isf['ISF_mask'] == file_isf.Nisf)#.chunk(chunks={'Nisf': Nisf_chunk}) 

In [ ]:
for tt in tqdm(range(yystart,yyend + 1)):
#for tt in tqdm(range(yystart,2063 + 1)):

    file_T_orig = xr.open_mfdataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc') #, chunks={'Nisf': Nisf_chunk})#, chunks={'Nisf': 1,'x': chunk_size, 'y': chunk_size})
    file_S_orig = xr.open_mfdataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc') #, chunks={'Nisf': Nisf_chunk})#, chunks={'Nisf': 1,'x': chunk_size, 'y': chunk_size})
    file_TS_orig = xr.merge([file_T_orig,file_S_orig])
    file_TS_orig = file_TS_orig.assign_coords({'z': -1*file_TS_orig['z'].values}).rename({'z':'depth'})
    file_TS = file_TS_orig.sel(Nisf=rignot_isf)#.chunk(chunks={'depth': 20, 'Nisf': 1})
    
    depth_axis_old = file_TS.depth.values
    depth_axis_new = np.concatenate((np.zeros(1),depth_axis_old))

    file_TS_with_shallow = file_TS.interp({'depth': depth_axis_new})
    filled_TS = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth') #, 'profile_domain': 1})

    # ice draft depth or deepest entrance depth
    depth_of_int0 = ice_draft_isf.where(ice_draft_isf<file_isf['front_bot_depth_max'], 
                                       file_isf['front_bot_depth_max'])
    depth_of_int = depth_of_int0.where(ice_draft_isf>file_isf['front_ice_depth_min'], 
                                       file_isf['front_ice_depth_min']) #.chunk(chunks={'Nisf': Nisf_chunk})
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] == file_isf.Nisf, 0)#.chunk({'Nisf': Nisf_chunk})

        
    #print('here1')
    T_isf = filled_TS['thetao'].interp({'depth': depth_of_int}).drop('depth')
    T_isf = T_isf.where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    T_isf = T_isf.to_dataset(name='theta_in')
    #print('here2')
    T_isf['salinity_in'] = filled_TS['so'].interp({'depth': depth_of_int}).drop('depth')
    T_isf['salinity_in'] = T_isf['salinity_in'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    #print('here3')
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] > 1).sum('Nisf')
    depth_of_int = depth_of_int.where(depth_of_int>0)
    #T_isf['freezing_T'] = meltf.freezing_temperature(T_isf['salinity_in'], -1*depth_of_int)
    #print('here4')
    #T_isf['thermal_forcing'] = T_isf['theta_in'] - T_isf['freezing_T']
    T_isf['depth_of_int'] = depth_of_int
    
    # project it on 2D
    #T_isf = T_isf.sum('Nisf')
    T_isf.assign_coords({'time':tt}).to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc','w')
    del T_isf
    del depth_of_int

In [ ]:
tt

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_allyy.nc')